In [7]:
import os
import operator
import numpy as np
import astropy
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy.coordinates import get_body

In [8]:
def get_nouns(filename):
    nouns = []
    with open(filename, 'r') as f:
        for line in f:
            nouns.append(line.strip())
    return nouns

In [9]:
t = Time("2014-09-22 23:22") # Observation time
loc = EarthLocation.of_site('greenwich') # Observation location

SolarSystemBodies = ['sun', 'mercury', 'venus', 'earth', 'moon', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune', 'pluto']

In [10]:
def search_object(object):
    # Convert to lower case
    object = object.lower()
    if object in SolarSystemBodies:
        # Get the RA and Dec of the object if in solar system
        with solar_system_ephemeris.set('builtin'): # Use for solar system bodies
            RADe_object = get_body(object, t, loc) 
            # Get ICRS coordinates
            RADe_object = RADe_object.icrs
    else:
        # A celestial object in ICRS outside the solar system
        RADe_object = astropy.coordinates.get_icrs_coordinates(object)
    
    return RADe_object

In [13]:
RADe_object = search_object('sun')
RA = RADe_object.ra.deg
DE = RADe_object.dec.deg
print('RA = ', RA)
print('DE = ', DE)

RA =  333.7965585063333
DE =  -13.653453396868821


In [14]:
RADe_object = search_object('M83')
RA = RADe_object.ra.deg
DE = RADe_object.dec.deg
print('RA = ', RA)
print('DE = ', DE)

RA =  204.25383
DE =  -29.865761111


In [15]:
# %pip install selenium # Install Selenium for feeding the web browser
# %pip install webdriver-manager # Install webdriver-manager for managing the web driver

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

ModuleNotFoundError: No module named 'selenium'

In [ ]:
browser.get("https://archive.stsci.edu/cgi-bin/dss_form?target=mars&resolver=SIMBAD")